In [1]:
from __future__ import print_function

from keras import backend as K
K.set_image_dim_ordering('th') # ensure our dimension notation matches

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, AveragePooling2D,Conv2DTranspose
from keras.layers.core import Flatten
from keras.optimizers import SGD, Adam
from keras.datasets import mnist
from keras import utils
import numpy as np
from PIL import Image, ImageOps
import argparse
import math

import os
import os.path

import glob

Using TensorFlow backend.


In [7]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*8*8))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((128, 8, 8), input_shape=(128*8*8,)))
    #model.add(UpSampling2D(size=(4, 4)))
    model.add(Conv2DTranspose(64, (5, 5), padding='same'))
    #model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(4, 4)))
    model.add(Conv2DTranspose(32, (5, 5), padding='same'))
    #model.add(Convolution2D(4, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(4, 4)))
    model.add(Conv2DTranspose(16, (5, 5),padding='same'))
    #model.add(Convolution2D(4, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    return model


def discriminator_model():
    model = Sequential()
    model.add(Convolution2D(
                        64, 5, 5,
                        border_mode='same',
                        input_shape=(4, 128, 128)))
    model.add(Activation('tanh'))
    model.add(AveragePooling2D(pool_size=(4, 4)))
    model.add(Convolution2D(128, 5, 5))
    model.add(Activation('tanh'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model


def generator_containing_discriminator(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model


def combine_images(generated_images):
    #print("combine_images start")
    #print(generated_images.shape)
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    
    #print(num)
    #print(width)
    #print(height)
    #print(shape)
    
    image = np.zeros((height*shape[0], width*shape[1],4),
                     dtype=generated_images.dtype)
    #print(image.shape)
    
    for index, img in enumerate(generated_images):
        #print(img.shape)
        i = int(index/width)
        j = index % width
        for d in range(1,img.shape[0] - 1):
            image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1],d] = \
                img[d, :, :]
    #print("combine_images end")
    #print(image.shape)
    #raise Exception("Testing")
    return image

In [8]:
model = discriminator_model()
print(model.summary())

/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same", input_shape=(4, 128, 1...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 128, 128)      6464      
_________________________________________________________________
activation_11 (Activation)   (None, 64, 128, 128)      0         
_________________________________________________________________
average_pooling2d_4 (Average (None, 64, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 14, 14)       204928    
_________________________________________________________________
activation_12 (Activation)   (None, 128, 14, 14)       0         
_________________________________________________________________
average_pooling2d_5 (Average (None, 128, 7, 7)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
__________

In [4]:
def load_data(pixels=128, verbose=False):
    print("Loading data")
    X_train = []
    paths = glob.glob(os.path.normpath(os.getcwd() + '/assets/*.jpg'))
    for path in paths:
        if verbose: print(path)
        im = Image.open(path)
        im = ImageOps.fit(im, (pixels, pixels), Image.ANTIALIAS)
        #im = ImageOps.grayscale(im)
        #im.show()
        im = np.asarray(im)
        #print(im.shape)
        X_train.append(im)
    print("Finished loading data")
    return np.array(X_train)

def train(epochs, BATCH_SIZE, weights=False):
    """
    :param epochs: Train for this many epochs
    :param BATCH_SIZE: Size of minibatch
    :param weights: If True, load weights from file, otherwise train the model from scratch. 
    Use this if you have already saved state of the network and want to train it further.
    """
    X_train = load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 4) + X_train.shape[1:3])
    
    #print(X_train.shape)
    discriminator = discriminator_model()
    generator = generator_model()
    if weights:
        generator.load_weights('goodgenerator.h5')
        discriminator.load_weights('gooddiscriminator.h5')
    discriminator_on_generator = \
        generator_containing_discriminator(generator, discriminator)
    #d_optim = SGD(lr=0.0001, momentum=0.9, nesterov=True)
    #g_optim = SGD(lr=0.0001, momentum=0.9, nesterov=True)
    d_optim = Adam(lr=0.0005)
    g_optim = Adam(lr=0.0005)
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    discriminator_on_generator.compile(
        loss='binary_crossentropy', optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)
    noise = np.zeros((BATCH_SIZE, 100))
    for epoch in range(epochs):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0)
            #print(generated_images.shape)
            if index % 20 == 0 and epoch % 10 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                destpath = os.path.normpath(os.getcwd()+ "/generated-images/"+str(epoch)+"_"+str(index)+".png")
                Image.fromarray(image.astype(np.uint8)).save(destpath)
            #print(image_batch.shape)
            #print(generated_images.shape)
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            discriminator.trainable = False
            g_loss = discriminator_on_generator.train_on_batch(
                noise, [1] * BATCH_SIZE)
            discriminator.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if epoch % 10 == 9:
                generator.save_weights('goodgenerator.h5', True)
                discriminator.save_weights('gooddiscriminator.h5', True)

def clean(image):
    print(image.shape)
    for d in range(1,image.shape[0] - 1):
        for i in range(1, image.shape[1] - 1):
            for j in range(1, image.shape[2] - 1):
                if image[d][i][j] + image[d][i+1][j] + image[d][i][j+1] + image[d][i-1][j] + image[d][i][j-1] > 127 * 5:
                    image[d][i][j] = 255
    return image
    

def generate(BATCH_SIZE):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('goodgenerator.h5')
    noise = np.zeros((BATCH_SIZE, 100))
    a = np.random.uniform(-1, 1, 100)
    b = np.random.uniform(-1, 1, 100)
    grad = (b - a) / BATCH_SIZE
    for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, 100)
    generated_images = generator.predict(noise, verbose=1)
    #image = combine_images(generated_images)
    print(generated_images.shape)
    for image in generated_images:
        #print(image.shape)
        #print(image[0])
        #print(image[1])
        #print(image[2])
        #image = image[0]
        image = image*127.5+127.5
        
        Image.fromarray(image, 'RGB').show()
        #Image.fromarray(image.astype(np.uint8)).save("dirty.png")
        Image.fromarray(image, 'RGB').save("dirty.png")
        #Image.fromarray(image.astype(np.uint8)).show()
        clean(image)
        #image = Image.fromarray(image.astype(np.uint8))
        image = Image.fromarray(image, 'RGB')
        image.show()        
        image.save("clean.png")



def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args

In [6]:
train(5, 10, False)

Loading data
Finished loading data


/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same", input_shape=(4, 128, 1...)`
/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5))`
/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=100)`
  app.launch_new_instance()
/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (5, 5), padding="same")`


Epoch is 0
Number of batches 5
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
(4, 128, 128)
batch 0 d_loss : 0.739809
batch 0 g_loss : 0.793575
batch 1 d_loss : 1.792831
batch 1 g_loss : 0.924381
batch 2 d_loss : 0.468558
batch 2 g_loss : 2.467206
batch 3 d_loss : 5.126073
batch 3 g_loss : 2.280564
batch 4 d_loss : 0.957758
batch 4 g_loss : 2.643013
Epoch is 1
Number of batches 5
batch 0 d_loss : 0.765423
batch 0 g_loss : 5.707515
batch 1 d_loss : 3.591858
batch 1 g_loss : 5.828582
batch 2 d_loss : 1.241703
batch 2 g_loss : 6.016473
batch 3 d_loss : 3.868869
batch 3 g_loss : 6.189105
batch 4 d_loss : 1.588841
batch 4 g_loss : 6.744358
Epoch is 2
Number of batches 5
batch 0 d_loss : 3.086124
batch 0 g_loss : 6.447855
batch 1 d_loss : 4.823086
batch 1 g_loss : 6.642921
batch 2 d_loss : 4.352969
batch 2 g_loss : 4.233600
batch 3 d_loss : 3.113111
batch 3 g_loss : 4.512159
batch 4 d_loss : 2.722462
batch 4 g_los

In [130]:
generate(1)

/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=100)`
  app.launch_new_instance()
/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
/Users/alcaljos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (5, 5), padding="same")`


1/1 [==============================] - 1s
(1, 4, 128, 128)
(4, 128, 128)
[[ 0.09858324  0.09316424  0.14849852 ...,  0.03664798  0.06538415
   0.00946514]
 [ 0.08641765  0.09341589  0.18521921 ...,  0.01526798  0.06291559
   0.04068746]
 [ 0.1649746   0.09792455  0.24829048 ...,  0.00211536  0.02975532
   0.03616983]
 ..., 
 [ 0.03507511  0.08513828  0.05603809 ..., -0.08382568 -0.15118948
  -0.1053988 ]
 [ 0.0101876   0.06303491  0.03877604 ..., -0.04542444 -0.08312324
  -0.07418858]
 [ 0.06179219  0.10074876  0.03616854 ..., -0.03644394 -0.07752594
  -0.03414172]]
[[ 0.03266602  0.01026591  0.09722574 ...,  0.09411719  0.06287338
   0.03825153]
 [ 0.01006349 -0.00991927  0.08109448 ...,  0.06742445  0.05379739
   0.04602179]
 [-0.13954741 -0.1593789  -0.05700313 ...,  0.0458615   0.05367853
   0.038512  ]
 ..., 
 [-0.17813499 -0.29598138 -0.27435863 ..., -0.02172759 -0.04021024
  -0.00704374]
 [-0.12090873 -0.20065887 -0.1809682  ..., -0.10875208 -0.14042926
  -0.06980611]
 [-0.10554

In [ ]:
from coremltools.converters.keras import convert
from keras.models import model_from_json

def convertToCoreMl():
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('goodgenerator.h5')
    model = model_from_json(generator.to_json())
    model.load_weights('goodgenerator.h5')

    coreml_model = convert(model)
    coreml_model.save('shoesGenerator.mlmodel')

convertToCoreMl()

